In [2]:
# # from functools import partial
from pathlib import Path
# from typing import Optional, Tuple
# import cv2
# import fire
import numpy as np
import torch
import torch.nn.functional as F
from accelerate import Accelerator
from PIL import Image
from scipy.sparse.linalg import eigsh
# from sklearn.cluster import KMeans, MiniBatchKMeans
# from sklearn.decomposition import PCA
# from torchvision.utils import draw_bounding_boxes
from tqdm import tqdm
import extract_utils as utils
from torch.utils.data import Dataset, DataLoader

import torchvision
from lightly.loss import NegativeCosineSimilarity
from lightly.loss import NegativeCosineSimilarity
from lightly.models.modules import SimSiamPredictionHead, SimSiamProjectionHead
from lightly.transforms import SimSiamTransform
from torch import nn

## Importing Features

In [3]:
features_dir="/home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/features/dino_vits16"
inputs = list(enumerate(sorted(Path(features_dir).iterdir())))
which_features='k'

In [4]:
# print("type of inputs=", type(inputs))
print("len of inputs=",len(inputs))
print("type of inputs[0]", type(inputs[0]))
print("length of inputs[0]=",len(inputs[0]))
print("inputs[0][0]=",inputs[0][0])
print("inputs[0][1]=", inputs[0][1])

len of inputs= 1000
type of inputs[0] <class 'tuple'>
length of inputs[0]= 2
inputs[0][0]= 0
inputs[0][1]= /home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/features/dino_vits16/0001.pth


In [5]:
feat_list=[]
for inp in tqdm(inputs):
    index, features_file = inp
    # print(index, features_file)
     # Load
    data_dict = torch.load(features_file, map_location='cpu')
    feats = data_dict[which_features].squeeze().cuda()
    # print(feats.shape)
    # tensor_features=torch.stack(torch.tensor(feats))
    feat_list.append(feats)
    # print("data_dict.keys=", data_dict.keys())
    # print('data_dict[k]:', data_dict['k'])
    # print('data_dict[k].shape=', data_dict['k'].shape)
    # print('data_dict[indices]:', data_dict['indices'])
    # print('data_dict[file]:', data_dict['file'])
    # print('data_dict[model_name]:', data_dict['model_name'])
    # print('data_dict[patch_size]:', data_dict['patch_size'])
    # print('data_dict[shape]:', data_dict['shape'])
    # print('type of data_dict[shape].shape=', type(data_dict['shape']))
    # print('data_dict[shape][0]:', data_dict['shape'][0])
    # print('data_dict[shape][1]:', data_dict['shape'][1])
    # print('data_dict[shape][2]:', data_dict['shape'][2])
    # print('data_dict[shape][3]:', data_dict['shape'][3])


100%|██████████| 1000/1000 [00:01<00:00, 626.91it/s]


## Custom Dataset on FeatureSpace

In [6]:
print(len(feat_list))
print(type(feat_list[0]))
print(feat_list[0].shape)

1000
<class 'torch.Tensor'>
torch.Size([400, 384])


In [7]:
# Define a custom dataset class
class Feature_Dataset(Dataset):
    def __init__(self, features):
        self.features = features

    def __getitem__(self, index):
        return self.features[index]

    def __len__(self):
        return len(self.features)

In [8]:
print(feat_list[1].shape)

torch.Size([400, 384])


In [9]:
# features=data_dict['k']
# print(features.shape)
feat_dataset = Feature_Dataset(feat_list[0])
if feat_list[0]%2!=0
    features_dataloader = DataLoader(feat_dataset, batch_size=2, shuffle=True, drop_last=True)
else:
    features_dataloader = DataLoader(feat_dataset, batch_size=2, shuffle=True)

SyntaxError: invalid syntax (3892473385.py, line 4)

In [ ]:
class SimSiam(nn.Module):
    def __init__(self):
        super().__init__()
        self.projection_head = SimSiamProjectionHead(feats.shape[1], 128,feats.shape[1])
        self.prediction_head = SimSiamPredictionHead(feats.shape[1], 128, feats.shape[1])

    def forward(self, x):
        z = self.projection_head(x)
        p = self.prediction_head(z)
        z = z.detach()
        return z, p

In [ ]:
model_simsiam = SimSiam()
device = "cuda" if torch.cuda.is_available() else "cpu"
model_simsiam.to(device)

In [ ]:
criterion = NegativeCosineSimilarity()
optimizer = torch.optim.SGD(model_simsiam.parameters(), lr=0.06)

In [ ]:
# print("Starting Training")
for epoch in range(10):
    total_loss = 0
    for x0 in features_dataloader:
    # for (x0), _, _ in features_dataloader:
        x0 = x0.unsqueeze(0).to(device)
        # print(x0.shape)
        x1=torchvision.transforms.RandomAffine(0)(x0)
        # print(x1.shape)
        x0 = x0.squeeze(0).to(device)
        # print("x0.shape=", x0.shape)
        x1 = x1.squeeze(0).to(device)
        # print("x1 shape=", x1.shape)

        z0, p0 = model_simsiam(x0)
        z1, p1 = model_simsiam(x1)
        loss = 0.5 * (criterion(z0, p1) + criterion(z1, p0))
        total_loss += loss.detach()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    avg_loss = total_loss / len(features_dataloader)
    print(f"epoch: {epoch:>02}, loss: {avg_loss:.5f}")

In [ ]:
projected_feature=model_simsiam(feat_list[1])

In [ ]:
print(type(projected_feature))
print(len(projected_feature))
print(projected_feature[0].shape)
print(projected_feature[1].shape)
print(projected_feature[1].t().shape)


In [ ]:
W=torch.matmul(projected_feature[0], projected_feature[1].t())

In [ ]:
print(W.shape)